## Generate a SWOT swath.

This example lets us understand how to initialize the simulation parameters (error, SSH interpolation, orbit), generate an orbit, generate a swath, interpolate the SSH, and simulate the measurement errors. Finally, we visualize the simulated data.

### Simulation setup

The configuration is defined using an associative dictionary between the expected parameters and the values of its parameters. The description of the parameters is available on the [online help](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.settings.Parameters.html#swot_simulator.settings.Parameters).

This array can be loaded from a Python file using the [eval_config_file](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.settings.eval_config_file.html#swot_simulator.settings.eval_config_file) method. But it can also be declared directly in Python code.

In [ ]:
import os

In [ ]:
configuration = dict(
    # The swath contains in its centre a central pixel divided in two by the
    # reference ground track.
    central_pixel = True,
    # Distance, in km, between two points along track direction.
    delta_al = 2.0,
    # Distance, in km, between two points across track direction.
    delta_ac = 2.0,
    # Distance, in km, between the nadir and the center of the first pixel of the
    # swath
    half_gap = 2.0,
    # Distance, in km, between the nadir and the center of the last pixel of the
    # swath
    half_swath = 70.0,
    # Limits of SWOT swath requirements. Measurements outside the span will be set
    # with fill values.
    requirement_bounds = [10, 60],
    # Ephemeris file to read containing the satellite's orbit.
    ephemeris=os.path.join("..", "data", "ephemeris_calval_june2015_ell.txt"),
    # Generation of measurement noise.
    noise=[
        'altimeter',
        'baseline_dilation',
        'karin',
        'roll_phase',
        'timing',
        'wet_troposphere',
    ],
    # File containing spectrum of instrument error
    error_spectrum=os.path.join("..", "data", "error_spectrum.nc"),
    # KaRIN file containing spectrum for several SWH
    karin_noise=os.path.join("..", "data", "karin_noise_v2.nc"),
    # The plug-in handling the SSH interpolation under the satellite swath.
    #ssh_plugin = TODO
)

In [ ]:
import swot_simulator.settings

parameters = swot_simulator.settings.Parameters(configuration)

We create the parameter object for our simulation.

### SSH interpolation

The written configuration allows us to simulate a swath. However, the interpolation of the SSH under the satellite swath remains undefined. If you don't need this parameter, you can skip this setting.

For our example, we use the SSH of the CMEMS grids provided on the Pangeo site.

In [ ]:
import intake

cat = intake.open_catalog("https://raw.githubusercontent.com/pangeo-data/"
                          "pangeo-datastore/master/intake-catalogs/master.yaml")

In [ ]:
ds = cat.ocean.sea_surface_height.to_dask()
ds

To interpolate SSH, we need to implement a class that must define a method to interpolate the data under the swath. This class must be derived from the [CartesianGridHandler](file:///Users/fbriol/Documents/workspace/swot_simulator/docs/build/html/generated/swot_simulator.plugins.CartesianGridHandler.html#swot_simulator.plugins.CartesianGridHandler) class to be correctly taken into account by the class managing the parameters.

In [ ]:
import pyinterp.backends.xarray
import numpy
import xarray
#
import swot_simulator.plugins


class CMEMS(swot_simulator.plugins.CartesianGridHandler):
    """
    Interpolation of the SSH AVISO (CMEMS L4 products).
    """
    def __init__(self, adt):
        self.adt = adt
        ts = adt.time.data

        assert numpy.all(ts[:-1] <= ts[1:])

        # The frequency between the grids must be constant.
        frequency = set(numpy.diff(ts.astype("datetime64[s]").astype("int64")))
        if len(frequency) != 1:
            raise RuntimeError(
                "Time series does not have a constant step between two "
                f"grids: {frequency} seconds")

        # The frequency is stored in order to load the grids required to
        # interpolate the SSH.
        self.dt = numpy.timedelta64(frequency.pop(), 'ns')

    def load_dataset(self, first_date, last_date):
        """Loads the 3D cube describing the SSH in time and space."""
        if first_date < self.adt.time[0] or last_date > self.adt.time[-1]:
            raise IndexError(
                f"period [{first_date}, {last_date}] is out of range: "
                f"[{self.adt.time[0]}, {self.adt.time[-1]}]")
        first_date = self.adt.time.sel(time=first_date, method='pad')
        last_date = self.adt.time.sel(time=last_date, method='backfill')
        selected = self.adt.loc[dict(time=slice(first_date, last_date))]
        selected = selected.compute()
        return pyinterp.backends.xarray.Grid3D(selected.adt)

    def interpolate(self, lon, lat, time):
        """Interpolate the SSH to the required coordinates"""
        interpolator = self.load_dataset(time.min(), time.max())
        ssh = interpolator.trivariate(dict(longitude=lon,
                                           latitude=lat,
                                           time=time),
                                      interpolator='bilinear')
        return ssh

Now we can update our parameters.

In [ ]:
parameters.ssh_plugin = CMEMS(ds)

### Initiating orbit propagator.

Initialization is simply done by [loading](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.orbit_propagator.load_ephemeris.html#swot_simulator.orbit_propagator.load_ephemeris) the ephemeris file. The satellite's one-day pass is taken into account in this case.

In [ ]:
import swot_simulator.orbit_propagator


with open(parameters.ephemeris, "r") as stream:
    orbit = swot_simulator.orbit_propagator.calculate_orbit(parameters, stream)

### Iterate on the half-orbits of a period.

To iterate over all the half-orbits of a period, call the method [iterate](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.orbit_propagator.Orbit.iterate.html#swot_simulator.orbit_propagator.Orbit.iterate).  This method returns all cycle numbers, trace numbers, and start dates of the half orbits within the period. If the start date remains not set, the method uses the current date. If the end date remains undefined, the method sets the end date to the start date plus the cycle duration.

In our case, we generate a cycle from January 1, 2000.

In [ ]:
first_date = numpy.datetime64("2000-01-01")
iterator = orbit.iterate(first_date)
cycle_number, pass_number, date = next(iterator)
cycle_number, pass_number, date

### Initialization of measurement error generators

Error initialization is done simply by calling the appropriate [class](https://swot-simulator.readthedocs.io/en/latest/generated/swot_simulator.error.generator.Generator.html#swot_simulator.error.generator.Generator). The initialization of the wet troposphere error generator takes a little time (about 40 seconds), which explains the processing time for the next cell.

In [ ]:
import swot_simulator.error.generator


error_generator = swot_simulator.error.generator.Generator(parameters, first_date)

### Generate the positions under the swath.

To perform this task, the following function is implemented.

> If the position of the pass is outside the area of interest (`parameters.area`),
> the generation of the pass can return `None`.

In [ ]:
def generate_one_track(pass_number, date, orbit):
    # Compute the spatial/temporal position of the satellite
    track = swot_simulator.orbit_propagator.calculate_pass(pass_number, orbit, parameters)

    # If the pass is not located in the area of interest (parameter.area)
    # the result of the generation can be null.
    if track is None:
        return None

    # Set the simulated date
    track.time = date
    
    return track

### Interpolate SSH

Interpolation of the SSH for the space-time coordinates generated by the simulator.

In [ ]:
def interpolate_ssh(parameters, track):
    swath_time = numpy.repeat(track.time, track.lon.shape[1]).reshape(track.lon.shape)
    ssh = parameters.ssh_plugin.interpolate(track.lon.flatten(),
                                            track.lat.flatten(),
                                            swath_time.flatten())
    return ssh.reshape(track.lon.shape)

### Calculation of instrumental errors

Simulation of instrumental errors. 

> Karin's instrumental noise can be modulated by wave heights.
> The parameter SWH takes either a constant or a matrix defining
> the SWH for the swath positions. 

In [ ]:
def generate_instrumental_errors(error_generator, cycle_number, pass_number, orbit, track):
    return error_generator.generate(cycle_number,
                                    pass_number,
                                    orbit.curvilinear_distance,
                                    track.time,
                                    track.x_al,
                                    track.x_ac,
                                    swh=2.0)

 ### Create the swath dataset
 
 Generation of the simulated swath. The function returns an xarray dataset for the half-orbit generated.

In [ ]:
import swot_simulator.product_specification


def generate_dataset(cycle_number,
                     pass_number,
                     track,
                     ssh,
                     noise_errors,
                     complete_product=False):
    product = swot_simulator.product_specification.Swath(track, central_pixel=True)
    # Mask to set the measurements outside the requirements of the mission to
    # NaN.
    mask = track.mask()
    ssh *= mask
    product.ssh(ssh + sum_error(noise_errors))
    product.simulated_true_ssh(ssh)
    for error in noise_errors.values():
        # Only the swaths must be masked
        if len(error.shape) == 2:
            error *= mask
    product.update_noise_errors(noise_errors)
    return product.to_xarray(cycle_number, pass_number, complete_product)

### Calculates the sum of the simulated errors.

In [ ]:
def sum_error(errors, swath=True):
    """Calculate the sum of errors"""
    dims = 2 if swath else 1
    return numpy.add.reduce(
        [item for item in errors.values() if len(item.shape) == dims])

### Swath generation.

Now we can combine the different components to generate the swath.

In [ ]:
import dask


def generate_swath(cycle_number, pass_number, date, parameters,
                   error_generator, orbit):
    client = dask.distributed.get_client()
    # Compute swath positions
    track = dask.delayed(generate_one_track)(pass_number, date, orbit)
    # Interpolate SSH
    ssh = dask.delayed(interpolate_ssh)(parameters, track)
    # Simulate instrumental errors
    noise_errors = dask.delayed(generate_instrumental_errors)(error_generator,
                                                              cycle_number,
                                                              pass_number,
                                                              orbit, track)
    # Finally generate the dataset
    return dask.delayed(generate_dataset)(
        cycle_number, pass_number, track, ssh, noise_errors,
        parameters.complete_product).compute()

The simulator calculation can be distributed on a Dask cluster.

In [ ]:
import dask.distributed

# A local cluster is used here.
cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
client

In [ ]:
error_generator_ = client.scatter(error_generator)
parameters_ = client.scatter(parameters)
orbit_ = client.scatter(orbit)

In [ ]:
future = client.submit(generate_swath,
                       cycle_number,
                       pass_number,
                       date,
                       parameters_,
                       error_generator_,
                       orbit_)
ds = client.gather(future)
ds

To calculate a trace set you can use the following code

    futures = []
    for cycle_number, pass_number, date in iterator:
        futures.append(client.submit(generate_swath,
                                     cycle_number,
                                     pass_number,
                                     date,
                                     error_generator_,
                                     orbit_,
                                     parameters))
    client.gather(futures)
    
### Visualization

In [ ]:
import matplotlib.pyplot
import cartopy.crs
import cartopy.feature
%matplotlib inline

Selection of a reduced geographical area for visualization.

In [ ]:
selected = ds.where((ds.latitude > -50) & (ds.latitude < -40), drop=True)

Simulated SSH measurements (Interpolated SSH and simulated instrumental errors).

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(24,12))
ax = fig.add_subplot(1, 1, 1, projection=cartopy.crs.PlateCarree())
contourf = ax.contourf(selected.longitude, selected.latitude, selected.ssh_karin,
            transform=cartopy.crs.PlateCarree(), levels=255, cmap='jet')
fig.colorbar(contourf, orientation='vertical')
ax.set_extent([60, 69, -50, -40], crs=cartopy.crs.PlateCarree())
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.COASTLINE)
ax.coastlines()

Simulated KaRIN instrumental noise.

In [ ]:
for item in selected.variables:
    if item.startswith("simulated_error"):
        variable = selected.variables[item]
        fig = matplotlib.pyplot.figure(figsize=(18, 8))
        ax = fig.add_subplot(1, 1, 1)
        image = ax.imshow(variable.T,
                          extent=[0, len(selected.num_lines), -70, 70],
                          cmap='jet')
        ax.set_title(variable.attrs['long_name'] + "(" +
                     variable.attrs['units'] + ")")
        ax.set_xlabel("num_lines")
        ax.set_ylabel("num_pixels")
        fig.colorbar(image,
                     orientation='vertical',
                     fraction=0.046 * 70 / 250,
                     pad=0.04)